# EDA 
> This notebook contains preprocessing and visualization of the texts in the data.

> For the visualization spacy, gensim and pyLDAvis is used.

> Spacy: Spacy is an library which is used for advance natural processing. It also contains tokenization, Name Entity Recognition(NER), Parts of Speech tagging(POS), etc

> Gensim: Gensim is used for unsupervised topic modelling and  finding similarity.

> pyLDAviz: It is used to visualize the topic in the LDA model that has been fitted to large corpus. It creates clusters of similar topics. Two or more clusters overlapping means that they are similar. You can change the number of clusters if you want.

In [ ]:
# Import the required tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import re
import pyLDAvis.gensim
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")

In [ ]:
train.head()

## Preprocessing

In [ ]:
# Check for data imbalance
sns.countplot(train["sentiment"]);

In [ ]:
# Check for missing values
train.isna().sum()

## Remove punctuations and numbers from the data

In [ ]:
train.dropna(inplace=True)
train = train.reset_index(drop=True)

In [ ]:
def clean_punc(text):
  cleaned_text = re.sub(r'[^\w\s]', '', text)
  cleaned_text = re.sub(r'[0-9]', r'', cleaned_text)
  return cleaned_text

In [ ]:
train["cleaned_text"] = np.vectorize(clean_punc)(train["text"])

In [ ]:
# Remove stopwords
stop_words = set(stopwords.words("english"))
train["cleaned_text"] = train["cleaned_text"].apply(lambda x : " ".join([w.lower() for w in x.split() if w not in stop_words and len(w) > 3]))

## Tokenize the tweet

In [ ]:
texts = train["cleaned_text"].apply(lambda x : x.split())

## Lemmatizing the text

In [ ]:
lemma = WordNetLemmatizer()
lemming_texts = texts.apply(lambda x:[lemma.lemmatize(i) for i in x])
lemming_texts

In [ ]:
for i in range(len(lemming_texts)):
  lemming_texts[i] = " ".join(lemming_texts[i])

train["cleaned_text"] = lemming_texts

In [ ]:
train

## Data Visualization

In [ ]:
positive_text = train["cleaned_text"][train["sentiment"]=="positive"].apply(lambda x : x.split())
negative_text = train["cleaned_text"][train["sentiment"]=="negative"].apply(lambda x: x.split())
neutral_text = train["cleaned_text"][train["sentiment"]=="neutral"].apply(lambda x:x.split())

In [ ]:
# Merge all the lists into one common list
positive_text = sum(positive_text, [])
negative_text = sum(negative_text, [])
neutral_text = sum(neutral_text, [])

In [ ]:
# Plot the most common words in positive sentiment
freq_pos = nltk.FreqDist(positive_text)
pos_df = pd.DataFrame({
    "words":list(freq_pos.keys()),
    "Count":list(freq_pos.values())
})
common_pos = pos_df.nlargest(columns="Count", n=30)
fig = px.bar(common_pos, x="words", y="Count", labels={"words": "Words", "Count":"Frequency"})
fig.show();

In [ ]:
# Plot the most common words in negative sentiment
freq_neg = nltk.FreqDist(negative_text)
neg_df = pd.DataFrame({
    "words":list(freq_neg.keys()),
    "Count":list(freq_neg.values())
})
common_neg = neg_df.nlargest(columns="Count", n=30)
fig = px.bar(common_neg, x="words", y="Count", labels={"words": "Words", "Count": "Frequency"})
fig.show();

In [ ]:
# Plot the most common words in neutral sentiment
freq_ntl = nltk.FreqDist(neutral_text)
ntl_df = pd.DataFrame({
    "words":list(freq_ntl.keys()),
    "Count":list(freq_ntl.values())
})
common_ntl = ntl_df.nlargest(columns="Count", n=30)
fig = px.bar(common_ntl, x="words", y="Count", labels={"words": "Words", "Count":"Frequency"})
fig.show();

## Wordcloud

In [ ]:
# Most common words in the texts
words = " ".join([text for text in train["cleaned_text"]])
wordclouds = WordCloud(width=900, height=600, random_state=42, max_font_size=110).generate(words)

plt.figure(figsize=(10, 10))
plt.imshow(wordclouds, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Wordcloud for positive words
positive_words = " ".join([word for word in train["cleaned_text"][train["sentiment"] == "positive"]])
wordcloud_pos = WordCloud(width=900, height=700, random_state=42, max_font_size=100).generate(positive_words)

plt.figure(figsize=(10, 10))
plt.imshow(wordcloud_pos)
plt.axis("off")
plt.show()

## Name Entity Recognition

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
ners = []
for text in train["cleaned_text"].values:
  doc = nlp(text)

  for entity in doc.ents:
    ners.append((entity.text, entity.label_))

In [ ]:
name_entity_df = pd.DataFrame(ners, columns = ["Entity Name", "Entity Label"])
entity_df = name_entity_df.groupby(by=["Entity Name", "Entity Label"]).size().sort_values(ascending=False).reset_index().rename(columns = {0: "Frequency"})

In [ ]:
# Plot the first 50 entities
figure = px.bar(x=entity_df["Entity Label"][:50], y=entity_df["Frequency"][:50]) 
figure.show()

## Topic Modelling

> Topic Modelling is the task of extracting the main topics from the documents.

>For this task LDA is used. LDA is used to classify the text in a document to a particular topic

In [ ]:
texts = train["cleaned_text"].apply(lambda x: x.split())
texts.head()

In [ ]:
# Build the dictionary of words from the document
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# Build the LDA Model 
import gensim
topics = 10
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=topics, id2word=dictionary, passes=15)
lda.save("LDA_model.gensim")

In [ ]:
display = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(display)